# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core.environment import Environment

from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
def download_data(url_zip, downloadpath, file):
    """Download and extract the training data."""
    import urllib
    from zipfile import ZipFile
    data_file = f'./{downloadpath}/archive.zip'
    download_url = 
    urllib.request.urlretrieve(download_url, filename=data_file)

    # extract files
    with ZipFile(data_file, 'r') as zip:
        print('extracting files...')
        zip.extractall()
        print('finished extracting')
        data_dir = zip.namelist()[0]

    # delete zip file
    os.remove(data_file)
    
    
    return data_dir

ws = Workspace.from_config()
zipurl = 'https://www.kaggle.com/rupeshraundal/marketcheck-automotive-data-us-canada/download'
filenm = 'ca-dealers-used.csv'
downloadpath = 'Data'
dskey = 'UsedAutoPrices'
dsdesc = 'Used Auto Price Training Data'
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    datapath = download_data(zipurl, downloadpath, filenm)

    # get the datastore to upload prepared data
    datastore = ws.get_default_datastore()

    # upload the local file from src_dir to the target_path in datastore
    datastore.upload(src_dir=downloadpath, target_path=downloadpath)
    dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, (os.join(downloadpath, filenm)))], separator=',', header=True, encoding='utf8')
    dataset = dataset.register(workspace=workspace,
                                 name=dskey,
                                 description=dsdesc)
    # dataset = Dataset.Tabular.from_delimited_files(os.join(datapath, filenm), separator=',', header=True, encoding='utf8')
    # https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py
    # https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#explore-data
    # https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-with-datasets
    
#########################################################

ws = Workspace.from_config()
experiment_name = 'PCAP_Exp_CarPrices_AutoML'
experiment=Experiment(ws, experiment_name)


amlcompute_cluster_name = "PCAP-Clus-DS12v2"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)





## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
        "n_estimators": uniform(10, 100),
        'max_Depth': choice(10, 2, None ),
        'criterion': choice("mse","squared_error"),
        'max_Features': choice('sqrt','log2',8,None) 
        }
)

#TODO: Create your estimator and hyperdrive config
# estimator = <your estimator here>

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets?view=azure-ml-py
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
# https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn?view=azure-ml-py
# AzureML-sklearn-0.24-ubuntu18.04-py37-cuda11-gpu
# myenv = Environment(workspace=ws, name="AzureML-Tutorial") 
myenv = Environment(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cuda11-gpu") 
dir_output = './outputs'
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--output_dir', dir_output, '--input_data', dskey],#dataset.as_named_input('UsedAutoPrices')],
                      compute_target=amlcomp,
                      environment=myenv)
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-parameter-tuning-with-hyperdrive.ipynb
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
hyperdrive_run_config = HyperDriveConfig(run_config=src,
                         hyperparameter_sampling=param_sampling,
                         policy=bandpolicy,
                         primary_metric_name="accuracy",
                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                         max_total_runs=100, 
                         max_concurrent_runs=5)

In [ ]:
#TODO: Submit your experiment

hdrive_run = exp.submit(hd_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
hdrive_run.wait_for_completion(show_output=True)
best_run = hdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n Number of estimators:',parameter_values[0])
print('\n Max Depth:',parameter_values[1])
print('\n Criterion:',parameter_values[2])
print('\n Max Features:',parameter_values[3])
print('\n FileNames:', best_run.get_file_names())

filename="BestHdriveModel"
#joblib.dump(value=best_run, filename=filename)



In [ ]:
#TODO: Save the best model
#This next line from here may be needed to actually store the model, but try without first
#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/keras/train-hyperparameter-tune-deploy-with-keras/train-hyperparameter-tune-deploy-with-keras.ipynb
#model = best_run.register_model(model_name='keras-mlp-mnist', model_path='outputs/model')
os.makedirs(f'./{filename}', exist_ok=True)
for f in best_run.get_file_names():
    if f.startswith(f'{dir_output}/model'):
        output_file_path = os.path.join(f'./model/{filename}', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)
        

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
myenv_inference = Environment(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference") 

model = best_run.register_model(model_name='RFHD-AutoPrices', model_path=f'{filename}/model/RF-AutoPrices_hdrive_model.joblib')


# Combine scoring script & environment in Inference configuration
dummy_inference_config = InferenceConfig(entry_script="score.py", environment=myenv_inference)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True)

ws_name = "SVC_AutoPrices"
service = Model.deploy(
    ws,
    ws_name,
    [model],
    inference_config = dummy_inference_config,
    deployment_config = deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)
primary, secondary = service.get_keys()
print(primary)
#Enable app insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

service = Webservice(workspace=ws, name=ws_name)


import requests
# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back

#Generally, I figure we would send in all the columns from the original dataset and 
#then run it through the clean function to whittle it down, but that's not necessary for this exercise.
data = {
    "data":
    [
        {
            'miles': "51000",
            'year': "2018",
            'make': "Ford",
            'model': "F-150",
            'engine_size': "5",
            'body_type': "Pickup",
            'vehicle_type': "Truck",
            'drivetrain': "4WD",
            'transmission': "Automatic",
            'fuel_type': "Unleaded / Unleaded",
            'state': "ON"
            #Should be around 44649
        }
    ,
        {
            'miles': "58900",
            'year': "2018",
            'make': "Toyota",
            'model': "Corolla",
            'engine_size': "1.8",
            'body_type': "Sedan",
            'vehicle_type': "Car",
            'drivetrain': "FWD",
            'transmission': "Automatic",
            'fuel_type': "Unleaded",
            'state': "AB"
            #Should be around 17850
        }
    ,
        {
            'miles': "56010",
            'year': "2018",
            'make': "Mercedes",
            'model': "GLC-Class",
            'engine_size': "2",
            'body_type': "SUV",
            'vehicle_type': "Truck",
            'drivetrain': "4WD",
            'transmission': "Automatic",
            'fuel_type': "Electric / Unleaded",
            'state': "QC"
            #Should be around 44995
        }
    ]
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics

print(service.get_logs())

service.delete()